# LIVRABLE 1 - Classification binaire (Projet LEYENDA)

# Objectif : Distinguer les photos naturelles (couleur ou noir et blanc) des autres types d'images (peintures, textes, dessins, schémas...)

CHANGER EN PNG, RESIZE
MODELE RGB
MODELE NB
RECONNAITRE PHOTO PEINTURE


In [ ]:
## Partie 1 : Initialisation & Vérification GPU
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import tensorflow as tf

# Vérification GPU
gpus = tf.config.list_physical_devices('GPU')
print("GPU disponible :", gpus)


In [ ]:
## Partie 2 : Organisation Initiale des Images
source_dir = "./images"
working_dir = "./images_work"

categories = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
os.makedirs(working_dir, exist_ok=True)

for category in categories:
    src_path = os.path.join(source_dir, category)
    dst_path = os.path.join(working_dir, category)

    if not os.path.exists(dst_path):
        shutil.copytree(src_path, dst_path)
        print(f"[COPIED] {category}")


In [ ]:
## Partie 3 : Conversion RGB et Redimensionnement
for category in categories:
    path = os.path.join(working_dir, category)
    files = [f for f in os.listdir(path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

    for file in tqdm(files, desc=f"Traitement {category}"):
        img_path = os.path.join(path, file)
        img = Image.open(img_path).convert('RGB').resize((256, 256), Image.Resampling.LANCZOS)
        img.save(os.path.join(path, Path(file).stem + '.png'))

        if not file.endswith('.png'):
            os.remove(img_path)
    print(f"[CONVERTED] {category}")

In [ ]:
## Partie 4 : Préparation Dataset binaire
binary_dir = os.path.join(working_dir, "binary_data")
photo_dir = os.path.join(binary_dir, "photo")
non_photo_dir = os.path.join(binary_dir, "non_photo")
os.makedirs(photo_dir, exist_ok=True)
os.makedirs(non_photo_dir, exist_ok=True)

for category in categories:
    src = os.path.join(working_dir, category)
    dst = photo_dir if category == "Photo" else non_photo_dir

    for file in os.listdir(src):

        shutil.copy(os.path.join(src, file), os.path.join(dst, f"{category}_{file}"))
    print(f"[COPIED] {category} images to {dst}")

In [78]:
## Partie 5 : Préparation avec Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = train_datagen.flow_from_directory(
    binary_dir, target_size=(256,256), batch_size=25, class_mode='binary', subset='training')

val_gen = train_datagen.flow_from_directory(
    binary_dir, target_size=(256,256), batch_size=25, class_mode='binary', subset='validation')
print(f"[TRAINING] {len(train_gen)} batches")
print(f"[VALIDATION] {len(val_gen)} batches")

Found 33120 images belonging to 2 classes.
Found 8279 images belonging to 2 classes.
[TRAINING] 1325 batches
[VALIDATION] 332 batches


In [79]:
## Partie 6 : Modèle CNN Optimisé
from tensorflow.keras import layers, models, regularizers

def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_42 (Conv2D)              │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_42 (MaxPooling2D) │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_43 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_44 (MaxPooling2D) │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,840,001 (56.61 MB)

 Trainable params: 14,839,553 (56.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
## Partie 7 : Entraînement et validation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
]

history = model.fit(train_gen, epochs=25, validation_data=val_gen, callbacks=callbacks)


Epoch 1/25
  64/1325 ━━━━━━━━━━━━━━━━━━━━ 6:44 320ms/step - accuracy: 0.7269 - loss: 40.4791

In [ ]:
## Partie 8 : Évaluation du modèle
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.legend()
plt.title('Loss')
plt.show()

# Matrice de confusion
val_labels = val_gen.classes
preds = model.predict(val_gen)
pred_classes = (preds > 0.5).astype("int32")

print(classification_report(val_labels, pred_classes, target_names=['non_photo','photo']))
ConfusionMatrixDisplay(confusion_matrix(val_labels, pred_classes), display_labels=['non_photo','photo']).plot(cmap='Blues')
plt.show()
